<a href="https://colab.research.google.com/github/la2015-hw/Group_10/blob/main/extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle --quiet

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mustansireranpurwala/sdss-image-dataset")

print("Path to dataset files:", path)


100%|██████████| 20.4G/20.4G [03:22<00:00, 108MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mustansireranpurwala/sdss-image-dataset/versions/1
